# Preamble

In [251]:
coin = 'bitcoin'

In [252]:
from pycoingecko import CoinGeckoAPI
gecko = CoinGeckoAPI()

import pandas as pd

import matplotlib.pyplot as plt
%matplotlib notebook

import datetime as dt

import numpy as np

In [253]:
def __get_historical_data_url(coin_id, currency_id, start_date, end_date):
    return 'https://www.coingecko.com/en/coins/{}/historical_data/{}?end_date={}&start_date={}'.format(coin_id, currency_id, end_date.isoformat(), start_date.isoformat()) 

def get_historical_data_scraper(coin_id, currency_id, start_date_s, end_date_s):
    start_date = dt.date.fromisoformat(start_date_s)
    end_date = dt.date.fromisoformat(end_date_s)

    url = __get_historical_data_url(coin_id, currency_id, start_date, end_date)
    r = requests.get(url, timeout=5, stream=True)
    
    try:
        dfs = pd.read_html(r.content, parse_dates=['Date'])
    except:
        return None
    if len(dfs) != 1:
        return None

    df = dfs[0][::-1].set_index("Date")
    df = df.replace({'[$,]': ''}, regex=True).apply(pd.to_numeric)
    df.columns = pd.MultiIndex.from_product([[coin_id], df.columns])

    return df

def get_historical_data_gecko(coin_id, currency_id, start_date_s, end_date_s):
    p = gecko.get_coin_market_chart_range_by_id(coin_id, currency_id, dt.datetime.fromisoformat(start_date_s).strftime("%s"), dt.datetime.fromisoformat(end_date_s).strftime("%s"))

    df_mcaps = pd.DataFrame(p['market_caps'], columns=['Date', 'Market Cap'])
    df_mcaps['Date'] = pd.to_datetime(df_mcaps['Date'].apply(lambda x: dt.datetime.utcfromtimestamp(x/1000).date()))
    df_mcaps.set_index("Date", inplace=True)

    df_vol = pd.DataFrame(p['total_volumes'], columns=['Date', 'Volume'])
    df_vol['Date'] = pd.to_datetime(df_vol['Date'].apply(lambda x: dt.datetime.utcfromtimestamp(x/1000).date()))
    df_vol.set_index("Date", inplace=True)

    df_prices = pd.DataFrame(p['prices'], columns=['Date', 'Open'])
    df_prices['Date'] = pd.to_datetime(df_prices['Date'].apply(lambda x: dt.datetime.utcfromtimestamp(x/1000).date()))
    df_prices.set_index("Date", inplace=True)

    df = pd.concat([df_mcaps, df_vol, df_prices], axis=1)
    df['Close'] = df['Open'].shift(-1)

    df.columns = pd.MultiIndex.from_product([[coin_id], df.columns])

    return df

def get_historical_data(coin_id, currency_id, start_date_s, end_date_s):
    df = None
    try:
        df = get_historical_data_gecko(coin_id, currency_id, start_date_s, end_date_s)
    except:
        try:
            df = get_historical_data_scraper(coin_id, currency_id, start_date_s, end_date_s)
        except:
            return None
    return df

# Data

In [254]:
# coins_list = pd.DataFrame(gecko.get_coins_list())

In [255]:
# list(x for x in coins_list['name'] if x.startswith('Syn'))
# coins_list.loc[coins_list['id'] == 'aave']

In [256]:
df = get_historical_data(coin, 'usd', '2013-01-01', '2021-04-01')
df_coin = df[coin]

In [257]:
# Monthly snapshot

index = pd.date_range(start='20130501', end='20210329', freq='MS', closed='left')
df_coin.loc[df_coin.index.isin(index)].head()

,Market Cap,Volume,Open,Close
Date,,,,
2013-05-01,1.298952e+09,0.0,117.000,103.430
2013-06-01,1.449500e+09,0.0,129.090,121.300
2013-07-01,1.107085e+09,0.0,97.504,89.489
2013-08-01,1.220482e+09,0.0,106.230,101.610
2013-09-01,1.607960e+09,0.0,138.170,137.170


In [258]:
df_coin_monthly = df_coin.loc['2013-05-01':].resample('MS').apply({'Open':'first', 'Close':'last', 'Volume':'sum', 'Market Cap':'last'})
df_coin_monthly['Change'] = (df_coin_monthly['Close'] - df_coin_monthly['Open'])/df_coin_monthly['Open']
df_coin_monthly['Log Change'] = np.log(df_coin_monthly['Close']) - np.log(df_coin_monthly['Open'])
df_coin_monthly.head()

,Open,Close,Volume,Market Cap,Change,Log Change
Date,,,,,,
2013-05-01,117.000,129.090,0.0,1.436387e+09,0.103333,0.098336
2013-06-01,129.090,97.504,0.0,1.082694e+09,-0.244682,-0.280616
2013-07-01,97.504,106.230,0.0,1.267183e+09,0.089494,0.085713
2013-08-01,106.230,138.170,0.0,1.602407e+09,0.300668,0.262878
2013-09-01,138.170,132.550,0.0,1.549942e+09,-0.040675,-0.041525


# Tables

Changes per month:

In [259]:
df_coin_monthly_change = df_coin_monthly['Change'].groupby(df_coin_monthly.index.year).apply(lambda x: pd.Series(x.values, index=x.index.month)).unstack()
df_coin_monthly_change

Date,1,2,3,4,5,6,7,8,9,10,11,12
Date,,,,,,,,,,,,
2013,NaN,NaN,NaN,NaN,0.103333,-0.244682,0.089494,0.300668,-0.040675,0.548548,4.034542,-0.257067
2014,0.084378,-0.327211,-0.145317,-0.049351,0.406667,0.005894,-0.077755,-0.198775,-0.199498,-0.149434,0.171348,-0.172593
2015,-0.279378,0.138940,-0.042420,-0.056874,-0.040485,0.153091,0.090078,-0.190730,0.046542,0.367052,0.118318,0.195287
2016,-0.145907,0.162597,-0.033885,0.085993,0.191289,0.256955,-0.106186,-0.056388,0.074301,0.187514,0.032707,0.325335
2017,-0.011500,0.244183,-0.115867,0.297803,0.729593,0.001553,0.122117,0.776338,-0.106024,0.544489,0.637538,0.281735
2018,-0.282335,0.022954,-0.325843,0.318685,-0.188953,-0.146601,0.215464,-0.091832,-0.057145,-0.039136,-0.370211,-0.079098
2019,-0.070572,0.111792,0.075457,0.289820,0.620246,0.269653,-0.075780,-0.045285,-0.139061,0.106457,-0.174329,-0.047806
2020,0.293657,-0.081120,-0.249186,0.340864,0.099450,-0.033510,0.237803,0.030619,-0.077229,0.279249,0.423183,0.480017
2021,0.139284,0.360062,0.239224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Log changes per month:

In [260]:
df_coin_monthly_log_change = df_coin_monthly['Log Change'].groupby(df_coin_monthly.index.year).apply(lambda x: pd.Series(x.values, index=x.index.month)).unstack()
df_coin_monthly_log_change

Date,1,2,3,4,5,6,7,8,9,10,11,12
Date,,,,,,,,,,,,
2013,NaN,NaN,NaN,NaN,0.098336,-0.280616,0.085713,0.262878,-0.041525,0.437318,1.616322,-0.297149
2014,0.081006,-0.396324,-0.157025,-0.050610,0.341223,0.005877,-0.080945,-0.221613,-0.222516,-0.161853,0.158155,-0.189458
2015,-0.327640,0.130098,-0.043346,-0.058556,-0.041327,0.142446,0.086249,-0.211623,0.045492,0.312656,0.111825,0.178386
2016,-0.157715,0.150656,-0.034472,0.082495,0.175036,0.228692,-0.112257,-0.058040,0.071671,0.171862,0.032183,0.281665
2017,-0.011566,0.218479,-0.123148,0.260673,0.547886,0.001552,0.115217,0.574554,-0.112076,0.434693,0.493194,0.248215
2018,-0.331752,0.022695,-0.394292,0.276635,-0.209429,-0.158528,0.195126,-0.096326,-0.058843,-0.039923,-0.462371,-0.082402
2019,-0.073186,0.105973,0.072746,0.254503,0.482578,0.238744,-0.078805,-0.046343,-0.149732,0.101163,-0.191559,-0.048987
2020,0.257473,-0.084600,-0.286597,0.293314,0.094810,-0.034085,0.213338,0.030159,-0.080374,0.246273,0.352896,0.392053
2021,0.130400,0.307530,0.214485,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Total change per month; i.e., what would have happened if you bought at the start of the month every time and sold at the start of the next month, every year.

In [261]:
print(list("{:.2f}%".format(x*100) for x in np.exp(df_coin_monthly_log_change.sum()) - 1))

['-35.14%', '57.54%', '-52.84%', '188.19%', '343.32%', '15.50%', '52.75%', '26.32%', '-42.18%', '349.15%', '725.36%', '61.98%']


Mean change per month

In [262]:
print(list("{:.2f}%".format(x*100) for x in np.exp(df_coin_monthly_log_change.mean()) - 1))

['-5.27%', '5.85%', '-8.97%', '16.32%', '20.46%', '1.82%', '5.44%', '2.96%', '-6.62%', '20.66%', '30.19%', '6.21%']
